## 5.4 Livelock
Multiple threads or processes are actively responding to each other to resolve conflict, but that prevents them from making progress. 
A livelock looks similar to a deadlock, in the sense that two threads are blocking each other for making progress, but the difference is that the threads in a livelock are actively trying to resolve the problem. A livelock can occur when two or more threads are designed to respond to the actions of each other. Both threads are busy doing something, but the combination of their efforts prevent them from actually making progress and accomplishing anything useful. The program will never reach the end. The ironic thing about livelocks is that they're often caused by algorithms that are intended to detect and recover from deadlock. If one or more processes or thread takes action to resolve the deadlock, then those threads can end up being overly polite and stuck in a livelock. To avoid that, ensure that only one process takes action chosen by priority or some other mechanism, like random selection.

In [2]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        second_chopstick.acquire()
        try:
            if sushi_count > 0:
                sushi_count -= 1
                print(name, 'took a piece! Sushi remaining:', sushi_count)
        finally:
            second_chopstick.release()
            first_chopstick.release()

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_c, chopstick_a)).start()


Barron took a piece! Sushi remaining: 499
Barron took a piece! Sushi remaining: 498
Barron took a piece! Sushi remaining: 497
Barron took a piece! Sushi remaining: 496
Barron took a piece! Sushi remaining: 495
Barron took a piece! Sushi remaining: 494
Barron took a piece! Sushi remaining: 493
Barron took a piece! Sushi remaining: 492
Barron took a piece! Sushi remaining: 491
Barron took a piece! Sushi remaining: 490
Barron took a piece! Sushi remaining: 489


We can see that after several iterations, the program hits a deadlock. The philosophers have acquired their first chopstick lock and they're stuck waiting for their second one to become available. One possible way to fix this problem would be to have the philosophers release the lock on their first chopstick if the second chopstick is not available when they try to take it. That will give another philosopher a chance to take that first chopstick and hopefully prevent a deadlock.

In [ ]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        if not second_chopstick.acquire(blocking=False):
            print(name, 'released their first chopstick.')
            first_chopstick.release()
        else:
            try:
                if sushi_count > 0:
                    sushi_count -= 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)
            finally:
                second_chopstick.release()
                first_chopstick.release()

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_c, chopstick_a)).start()


If we run this program, we will see a lot of output messages that the philosophers are releasing their first chopstick. Occasionally, the scheduling might work out so that one of the philosophers is able to acquire both of the locks they need to take a bite of sushi, but the majority of the time these philosophers are picking up their first chopstick, seeing that the second one is not available, so they politely put the first chopstick back, so another philosopher can use it. All of the threads are actively working to pick up and put down chopsticks. They're just not making any useful progress.

Livelocks can be even harder to locate and debug than deadlocks, but when you multi-threaded program seems to be stuck in a lock, looking at the CPU utilization may give you some insight into whether it's a live or deadlock.

To resolve this livelock, we're going to introduce some randomness into the locking progress by importing the `time` and `random` modules. Then after a philosopher decides to put back their first chopstick, we'll make them sleep/philosophize for a random amount of time.

In [ ]:
#!/usr/bin/env python3
""" Three philosophers, thinking and eating sushi """

import threading
import time
from random import random

chopstick_a = threading.Lock()
chopstick_b = threading.Lock()
chopstick_c = threading.Lock()
sushi_count = 500

def philosopher(name, first_chopstick, second_chopstick):
    global sushi_count
    while sushi_count > 0: # eat sushi until it's all gone
        first_chopstick.acquire()
        if not second_chopstick.acquire(blocking=False):
            print(name, 'released their first chopstick.')
            first_chopstick.release()
            time.sleep(random()/10)
        else:
            try:
                if sushi_count > 0:
                    sushi_count -= 1
                    print(name, 'took a piece! Sushi remaining:', sushi_count)
            finally:
                second_chopstick.release()
                first_chopstick.release()

if __name__ == '__main__':
    threading.Thread(target=philosopher, args=('Barron', chopstick_a, chopstick_b)).start()
    threading.Thread(target=philosopher, args=('Olivia', chopstick_b, chopstick_c)).start()
    threading.Thread(target=philosopher, args=('Steve', chopstick_c, chopstick_a)).start()


If we run the program, we'll see way fewer instances that the philosophers returned their chopsticks. So they're able to keep taking pieces of sushi. The program makes progress and is able to finish without running into a deadlock or livelock.